In [1]:
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
from smodels.decomposition.theorySMS import TheorySMS
from smodels.experiment.expSMS import ExpSMS
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.experiment.defaultFinalStates import finalStates
from smodels.base.model import Model
from collections import OrderedDict
from smodels.base.physicsUnits import fb, GeV
from itertools import product
from smodels.experiment.databaseObj import Database
import copy
import re
from smodels.base.smodelsLogging import logger
import plotly.graph_objects as go
import networkx as nx
import numpy as np
logger.setLevel('ERROR')

### Load database

In [2]:
db = Database('official')

### Build dictionary with unique SMS and their links to analyses

In [3]:
databaseSMSDict = {}
uniqueSMSDict = {}
allSMS = []
for iexp,exp in enumerate(db.getExpResults()):
    expID = exp.globalInfo.id
    for ids,dataset in enumerate(exp.datasets):
        dsID = dataset.dataInfo.dataId
        for itx,tx in enumerate(dataset.txnameList):
            txID = str(tx)
            tx.sortSMSMap()
            for sms,smsLabel in tx.smsMap.items():
                allSMS.append(sms)
                smsMatch = None
                for sms1 in databaseSMSDict:
                    if sms.identicalTo(sms1):
                        smsMatch = sms1
                        break
                                
                # Update dictionary
                if smsMatch is None:
                    databaseSMSDict[sms] = {iexp : {ids : {itx : smsLabel}}}
                    uniqueSMSDict[sms] = {expID : {dsID : {txID : smsLabel}}}
                else:
                    useDict = databaseSMSDict[smsMatch]
                    entry = [iexp,ids,itx]
                    for key in entry:
                        if useDict.get(key) is None:
                            useDict.update({key : {}})
                        useDict = useDict.get(key)
                    useDict[itx] = smsLabel

                    if not expID in uniqueSMSDict[smsMatch]:
                        uniqueSMSDict[smsMatch].update({expID : {}})
                    if not dsID in  uniqueSMSDict[smsMatch][expID]:
                        uniqueSMSDict[smsMatch][expID].update({dsID : {}})
                    uniqueSMSDict[smsMatch][expID][dsID] = {txID : smsLabel}
                    

## Create Graph Map

### Restrict SMS dictionary to first few entries and define meaningful labels

In [4]:
# Restrict dictionary:
gDict = {'SMS%i' %i : value for i,value in enumerate(list(databaseSMSDict.values())[:3])}
labelsDict = {}
for isms,(sms,expDict) in enumerate(databaseSMSDict.items()):
    smsLabel = 'SMS%i' %isms
    txNames = set([])
    for iexp,dsDict in expDict.items():
        exp = db.getExpResults()[iexp]
        expLabel = exp.globalInfo.id
        labelsDict[iexp] = expLabel
        for ids,txDict in dsDict.items():
            for itx in txDict:
                tx = exp.datasets[ids].txnameList[itx]
                txNames.add(str(tx))
    labelsDict[smsLabel] = ','.join(sorted(list(txNames)))


### Generate Graph object

In [5]:
G = nx.from_dict_of_dicts(gDict)

### Collect necessary data for plotting

In [6]:
# Position using kamada weights
pos = nx.kamada_kawai_layout(G)

node_x = []
node_y = []
labels = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    if isinstance(node,str):
        labels.append(labelsDict[node])
    else:
        labels.append(None)

hover_text = []
node_adjacencies = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_name = list(G.nodes())[node]
    if isinstance(node_name,str):
        hover_text.append('%i results'%(len(adjacencies[1])))
    else:
        hover_text.append('%s'%(labelsDict[node_name]))

edge_x = []
edge_y = []

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

#### Create relevant scatter plots for edges and nodes

In [7]:
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')



node_trace = go.Scatter(
    x=node_x, y=node_y,
    textfont = {'size' : 20},
    mode='markers+text',
    customdata=hover_text,
    hovertemplate='%{customdata}<extra></extra>',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='SMS Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

node_trace.marker.color = node_adjacencies
node_trace.marker.size = [max(v,15) for v in node_adjacencies]
node_trace.text = labels        

### Generate interactive plot

In [8]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Database v%s' %db.databaseVersion,
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()